In [ ]:
import socceraction.spadl as spadl
from socceraction.data.statsbomb import StatsBombLoader
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import os
api = StatsBombLoader(getter="local", root="open-data-master/data")


In [ ]:

FL, FW = 105.0, 68.0  # meters

def sb_to_spadl_xy(x, y, fidelity_version=None, assume_cell_center=False):
    """
    Convert a single StatsBomb (x,y) to SPADL meters.
    - If `assume_cell_center=True`, subtract half-cell. Use for old, integer-ish event coords.
    - For 360 freeze-frames, pass assume_cell_center=False (no center shift).
    """
    if assume_cell_center:
        cell_side = 0.1 if fidelity_version == 2 else 1.0
        x = x - cell_side/2.0
        y = y - cell_side/2.0

    x_m = np.clip(x / 120.0 * FL, 0, FL)
    y_m = np.clip(FW - (y / 80.0 * FW), 0, FW)
    return x_m, y_m

def ltr_flip_if_away(x_m, y_m, is_away):
    if is_away:
        return (FL - x_m, FW - y_m)
    return (x_m, y_m)

# Example: transform a 360 freeze-frame list `ff` for the same LTR convention as SPADL
def transform_freeze_frame(ff, is_away, fidelity_version=None):
    out = []
    for p in ff:
        x, y = p["location"]
        xm, ym = sb_to_spadl_xy(x, y, fidelity_version, assume_cell_center=True)  # key difference
        xm, ym = ltr_flip_if_away(xm, ym, is_away)
        q = dict(p)
        q["location_spadl"] = [xm, ym]
        out.append(q)
    return out




In [ ]:
game_ids = []
for entry in os.scandir("open-data-master/data/three-sixty"):
    if entry.is_file():
        game_ids.append(int(entry.name.split('.')[0]))

print(f'Example Game IDs: {game_ids[3:6]}')
print(f'Total Games: {len(game_ids)}')

example_events = api.events(game_id=3788742,load_360=True)


In [ ]:
print(example_events.freeze_frame_360.tolist()[15])

In [ ]:

event = example_events[7:8]

three_sixty_data = event.freeze_frame_360.item()
location = event.location.item()
team_id = event.team_id.item()

for player in three_sixty_data:
    if player['actor'] == True:
        print(f"Player Loation (360 Data): {player['location']}")
        print(f"Player Loation (Reg Data): {location}")
        print(f"Player Team ID: {team_id}")
        print(f"Time: {event.timestamp.item()}")



In [ ]:
spadl_action = spadl.statsbomb.convert_to_actions(event, home_team_id=776)
spadl_action = (
  spadl
  .add_names(spadl_action)  # add actiontype and result names
)
fidelity = spadl.statsbomb._infer_xy_fidelity_versions(event)[1]

spadl_action_l2r = spadl.play_left_to_right(spadl_action, home_team_id=776)

spadl_location = [spadl_action_l2r.start_x.item(), spadl_action_l2r.start_y.item()]
spadl_end_location = [spadl_action_l2r.end_x.item(), spadl_action_l2r.end_y.item()]

transformed_threesixty = transform_freeze_frame(three_sixty_data,is_away=False,fidelity_version=fidelity) 

#spadl_action.type_name.item() == 'pass'




In [ ]:

for idx,id in enumerate(game_ids):
    try:
        events = api.events(game_id = id, load_360 = True)
    except:
        continue

    for idx,event in events.iterrows():
        print(event.type_id)
        if event.type_id == 0:
            print('PASS')
        
        #elif event.type_id == 0:
        #    print(event.type_id)
        
        """three_sixty = events.freeze_frame_360.to_list()
    
        spadl_action = spadl.statsbomb.convert_to_actions(event, home_team_id=team_id)
        spadl_action = spadl.add_names(spadl_action) # add actiontype and result names
        spadl_action_l2r = spadl.play_left_to_right(spadl_action, home_team_id=team_id)"""

    
    
        
    


In [13]:
spadl_action = spadl.statsbomb.convert_to_actions(event, home_team_id=776)
spadl_action = (
  spadl
  .add_names(spadl_action)  # add actiontype and result names
  .merge(api.teams(game_id=3788742))  # add team names
  .merge(api.players(game_id=3788742))  # add player names
)
fidelity = spadl.statsbomb._infer_xy_fidelity_versions(event)[1]

spadl_action_l2r = spadl.play_left_to_right(spadl_action, home_team_id=776)

spadl_location = [spadl_action_l2r.start_x.item(), spadl_action_l2r.start_y.item()]
spadl_end_location = [spadl_action_l2r.end_x.item(), spadl_action_l2r.end_y.item()]

transformed_threesixty = transform_freeze_frame(three_sixty_data,is_away=False,fidelity_version=fidelity)

for player in transformed_threesixty:
    if player['actor'] == True:
        threesixty_location = player['location_spadl']
        print(f"Player Loation (Transformed 360 Data): {threesixty_location}")
        print(f"Player Loation (Spadl 360 Data): {spadl_location}")
        print(f"Player Team ID: {team_id}")
        print(f"Time (360 Data): {event.timestamp.item()}")
        print(f"Time (Spadl Data): {spadl_action.time_seconds.item()}")





AttributeError: 'list' object has no attribute 'apply'